In [1]:
import os
import sys
import math
import numpy as np
import pandas as pd

In [ ]:
# import data

In [4]:
atts = {'color': ['L', 'C', 'H'], 'firmness': 'FIRMEZA', 'color_p': ['LP', 'CP', 'HP'], 'sst': 'SST', \
        'acidity': 'TOTAL ACIDITY', 'dry mass': 'MASSA SECA'}

# PLS only

In [14]:
list(atts.keys())[0]

'color'

In [ ]:
for att in list(atts.keys():
    y_vars = atts[att]